Ingest and pull airline data 

In [4]:
import json
import sqlite3
import random
from datetime import datetime, timedelta
import pprint

random.seed(42)

In [5]:
locations = ['JFK', 'LAX', 'ORD', 'ATL', 'DFW', 'DEN', 'SFO', 'SEA', 'MIA', 'BOS']
airline_codes = ['AA', 'DL', 'UA', 'SW', 'JB', 'AL']

In [6]:

def generate_flight_data(num_records=5000):
    flight_data = []
    base_time = datetime(2025, 6, 18, 5, 0, 0)

    for _ in range(num_records):
        departure_location = random.choice(locations)
        arrival_location = random.choice([loc for loc in locations if loc != departure_location])

        departure_time = base_time + timedelta(minutes=random.randint(0, 60*24*30))  # within next 30 days
        flight_duration = timedelta(minutes=random.randint(60, 300))  # 1–5 hour flight
        arrival_time = departure_time + flight_duration

        flight_number = f"{random.choice(airline_codes)}{random.randint(100, 9999)}"

        flight_data.append({
            "flight_number": flight_number,
            "departure_location": departure_location,
            "arrival_location": arrival_location,
            "departure_time": departure_time.strftime('%Y-%m-%d %H:%M:%S'),
            "arrival_time": arrival_time.strftime('%Y-%m-%d %H:%M:%S')
        })

    return flight_data

In [ ]:
flight_data = generate_flight_data()

# Write to a JSON file
with open("synthetic_airline_data.json", "w") as f:
    json.dump(synthetic_data, f, indent=2)

print("Synthetic airline data (5,000 records) saved to 'synthetic_airline_data.json'")


In [ ]:
pprint.pprint(flight_data[:5])

In [ ]:
conn = sqlite3.connect("flights.db")
c = conn.cursor()

c.execute("""
CREATE TABLE IF NOT EXISTS flights (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    flight_number TEXT NOT NULL,
    departure_location TEXT NOT NULL,
    arrival_location TEXT NOT NULL,
    departure_time TEXT NOT NULL,
    arrival_time TEXT NOT NULL
)
""")

for flight in flight_data:
    c.execute("""
    INSERT INTO flights (flight_number, departure_location, arrival_location, departure_time, arrival_time)
    VALUES (?, ?, ?, ?, ?)
    """, (
        flight["flight_number"],
        flight["departure_location"],
        flight["arrival_location"],
        flight["departure_time"],
        flight["arrival_time"]
    ))

conn.commit()
conn.close()


## augment the synthetic data tables

In [2]:
import pandas as pd
import numpy as np
import sqlite3

# Load flights
flights = pd.read_csv("united_ops/cleaned_flights.csv")
delayed_or_cancelled = flights[flights["status"].isin(["delayed", "cancelled"])].copy()

# Regenerate CREW
crew_rows = []
for _, row in delayed_or_cancelled.iterrows():
    n_crew = np.random.randint(4, 7)
    for i in range(n_crew):
        crew_rows.append({
            "crew_id": f"{row.flight_number}_C{i}",
            "name": f"Crew_{i}",
            "assigned_flight": row.flight_number,
            "base": "ORD",
            "duty_start": row["departure_time"],
            "duty_end": row["arrival_time"],
        })
crew_df = pd.DataFrame(crew_rows)

# Regenerate PASSENGERS
passenger_rows = []
for _, row in delayed_or_cancelled.iterrows():
    n_pax = np.random.randint(10, 151)
    for i in range(n_pax):
        passenger_rows.append({
            "passenger_id": f"{row.flight_number}_P{i}",
            "name": f"Passenger_{i}",
            "flight_number": row.flight_number,
            "loyalty_tier": np.random.choice(["Basic", "Silver", "Gold", "1K"], p=[0.6, 0.2, 0.15, 0.05]),
        })
passengers_df = pd.DataFrame(passenger_rows)

# Save and overwrite DB
crew_df.to_csv("united_ops/cleaned_crew.csv", index=False)
passengers_df.to_csv("united_ops/cleaned_passengers.csv", index=False)

conn = sqlite3.connect("united_ops.db")
crew_df.to_sql("crew", conn, if_exists="replace", index=False)
passengers_df.to_sql("passengers", conn, if_exists="replace", index=False)
conn.commit()
conn.close()


### Create Tables

In [5]:
import pandas as pd
import sqlite3

crew_df = pd.read_csv("united_ops/cleaned_crew.csv")
flights_df = pd.read_csv("united_ops/cleaned_flights.csv")
passengers_df = pd.read_csv("united_ops/cleaned_passengers.csv")

conn = sqlite3.connect("united_ops.db")

crew_df.to_sql("crew", conn, if_exists="replace", index=False)
flights_df.to_sql("flights", conn, if_exists="replace", index=False)
passengers_df.to_sql("passengers", conn, if_exists="replace", index=False)

conn.execute("CREATE INDEX IF NOT EXISTS idx_flights_flight_number ON flights(flight_number);")
conn.execute("CREATE INDEX IF NOT EXISTS idx_passengers_flight_number ON passengers(flight_number);")
conn.execute("CREATE INDEX IF NOT EXISTS idx_crew_flight ON crew(assigned_flight);")

conn.commit()
conn.close()


### Creating a Test Scenario

In [6]:
def get_delay_issues_from_db(db_path="united_ops.db") -> pd.DataFrame:
    """
    Fetch delayed or cancelled flights with crew and passenger counts from the United Ops DB.
    
    Args:
        db_path (str): Path to the SQLite database file.

    Returns:
        pd.DataFrame: Table of delay/cancellation issues with relevant operational info.
    """
    query = """
    SELECT 
        f.flight_number AS "Flight",
        f.departure_location AS "From",
        f.arrival_location AS "To",
        f.status AS "Status",
        f.departure_time AS "Dep Time",
        f.arrival_time AS "Arr Time",
        f.gate AS "Gate",
        COUNT(DISTINCT c.crew_id) AS "Crew Assigned",
        COUNT(DISTINCT p.passenger_id) AS "Passenger Count"
    FROM flights f
    LEFT JOIN crew c ON f.flight_number = c.assigned_flight
    LEFT JOIN passengers p ON f.flight_number = p.flight_number
    WHERE f.status IN ('delayed', 'cancelled')
    GROUP BY f.flight_number
    ORDER BY f.departure_time;
    """

    with sqlite3.connect(db_path) as conn:
        return pd.read_sql_query(query, conn)

In [7]:
delay_df = get_delay_issues_from_db("united_ops.db")
print(delay_df.head())

   Flight From   To     Status             Dep Time             Arr Time Gate  \
0  JB6515  ORD  ATL  cancelled  2025-06-18 06:57:00  2025-06-18 10:56:00  B11   
1   AL895  ORD  MIA    delayed  2025-06-18 09:44:00  2025-06-18 12:59:00  B18   
2  AA1132  SFO  ORD  cancelled  2025-06-18 19:29:00  2025-06-18 22:56:00  B10   
3   AA360  ORD  ATL  cancelled  2025-06-18 20:09:00  2025-06-18 21:57:00   B2   
4  JB8228  SFO  ORD    delayed  2025-06-18 21:17:00  2025-06-18 22:35:00   B5   

   Crew Assigned  Passenger Count  
0              5              107  
1              4               71  
2              6               80  
3              6               68  
4              4              129  
